In [1]:
from keras.models import Model
from keras.layers import Embedding, Conv1D, Dense, TimeDistributed, Dropout, Bidirectional, LSTM, Concatenate, Input
from keras_contrib.layers import CRF
from gensim.models import Word2Vec
import numpy as np

# Load pre-trained Word2Vec model
word2vec_model = Word2Vec.load('word2vec_model.bin')

# Define hyperparameters
input_dim = len(word2vec_model.wv.vocab)
output_dim = 100
max_seq_len = 100
num_labels = 3
filter_sizes = [3, 4, 5]
num_filters = 100
lstm_units = 128
dropout_rate = 0.5

# Define input layer
input_layer = Input(shape=(max_seq_len,))

# Define embedding layer
embedding_layer = Embedding(input_dim=input_dim,
    output_dim=output_dim,
    input_length=max_seq_len,
    weights=[np.array([word2vec_model[w] if w in word2vec_model else np.zeros(output_dim) for w in word2vec_model.wv.vocab])],
    trainable=False
)(input_layer)

# Define convolutional layers
conv_layers = []
for filter_size in filter_sizes:
    conv_layer = Conv1D(filters=num_filters,
                        kernel_size=filter_size,
                        activation='relu')(embedding_layer)
    conv_layer = Dropout(dropout_rate)(conv_layer)
    conv_layer = TimeDistributed(Dense(lstm_units, activation='relu'))(conv_layer)
    conv_layers.append(conv_layer)

# Concatenate convolutional layers
concat_layer = Concatenate()(conv_layers)

# Define BiLSTM layer
bi_lstm_layer = Bidirectional(LSTM(units=lstm_units,
                                    return_sequences=True,
                                    dropout=dropout_rate,
                                    recurrent_dropout=dropout_rate))(concat_layer)

# Define CRF layer
crf_layer = CRF(num_labels)

# Define output layer
output_layer = crf_layer(bi_lstm_layer)

# Define model
model = Model(inputs=input_layer, outputs=output_layer)
# model.compile(optimizer='adam', loss=crf_layer.loss_function, metrics=[crf_layer.accuracy])

# # Train model
# model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

model.summary()

FileNotFoundError: [Errno 2] No such file or directory: 'word2vec_model.bin'